[HR competition](https://www.hackerrank.com/contests/machine-learning-codesprint/challenges/hackerrank-predict-email-opens). Evaluation is done using [F1 score](https://en.wikipedia.org/wiki/F1_score).

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML, clear_output
from sklearn.metrics import f1_score as f1

pd.set_option('display.max_columns', None)

from sklearn import preprocessing, cross_validation
from sklearn.grid_search import GridSearchCV
from sklearn import linear_model, svm, neighbors, ensemble, naive_bayes
from datetime import datetime

In [2]:
df_1 = pd.read_csv('input/training_dataset.csv')
df_2 = pd.read_csv('input/test_dataset.csv')

Binary classification problem on the unbalanced dataset 0.33%.

Only `mail_category`, `last_online`, `hacker_timezone` have missing data. The amount of missing is very small:
 - mail_category < 0.001
 - last_online < 0.001
 - hacker_timezone ~ 1%

A couple of fields that are categorical, but presented as strings:
 - `mail_category`. Training (17 + null), test (14 + null).
 - `mail_type`. Training test has (4 + null), test has (1 + null). Will remove it.
 
Also there are a few timestamp fields. By themself they are useless. Will extract some data from their deltas. **May be try to extract month/day when email was sent**.

In [3]:
def convert(df):
    df['mail_category'] = pd.to_numeric(df['mail_category'].str.split('y_').str.get(-1))
    
    timestamp_fields = ['sent_time', 'last_online', 'hacker_created_at']
    for el in timestamp_fields:
        df[el] = df[el].astype('datetime64[s]')
        
    df['age_sent'] = (df['sent_time'] - df['hacker_created_at']) / np.timedelta64(1, 'D')
    df['last_seen'] = (df['sent_time'] - df['last_online']) / np.timedelta64(1, 'D')
    
    df.drop(['mail_type'] + timestamp_fields, axis=1, inplace=True)

In [4]:
df_1.drop(['click_time', 'clicked', 'open_time', 'unsubscribe_time', 'unsubscribed'], axis=1, inplace=True)

convert(df_1)
convert(df_2)

The idea of anonymizing `mail_id` and `user_id` as a long string (most probably `base64(hash)`) is pretty stupid and inefficient. Will convert them to integers to save space. I got **31,440** unique userIds and **164** mail campains

In [5]:
user_ids = pd.concat([df_1['user_id'], df_2['user_id']]).unique().tolist()
mail_ids = pd.concat([df_1['mail_id'], df_2['mail_id']]).unique().tolist()

user_map = {v: k for k, v in enumerate(user_ids)}
mail_map = {v: k for k, v in enumerate(mail_ids)}

In [6]:
df_1['user_id'] = df_1['user_id'].apply(lambda x: user_map[x])
df_2['user_id'] = df_2['user_id'].apply(lambda x: user_map[x])

df_1['mail_id'] = df_1['mail_id'].apply(lambda x: mail_map[x])
df_2['mail_id'] = df_2['mail_id'].apply(lambda x: mail_map[x])

### Add open rate

In [7]:
gb = df_1[['user_id', 'opened']].groupby('user_id')
user_info = (gb.sum() / gb.count()).reset_index()
user_info.columns = ['user_id', 'opened_rate']

df_1 = df_1.merge(user_info, on='user_id')

Now the dataframes are only numeric. There are some boolean and some categorical, but no strings. I do not like very long names and the order of the columns.

In [8]:
rename_dict = {
    'contest_login_count'          : 'clc_all',
    'contest_login_count_1_days'   : 'clc_1',
    'contest_login_count_7_days'   : 'clc_7',
    'contest_login_count_30_days'  : 'clc_30',
    'contest_login_count_365_days' : 'clc_365',
    
    'contest_participation_count'           : 'cpc_all',
    'contest_participation_count_1_days'    : 'cpc_1',
    'contest_participation_count_7_days'    : 'cpc_7',
    'contest_participation_count_30_days'   : 'cpc_30',
    'contest_participation_count_365_days'  : 'cpc_365',
    
    'submissions_count'         : 'subm_all',
    'submissions_count_1_days'  : 'subm_1',
    'submissions_count_7_days'  : 'subm_7',
    'submissions_count_30_days' : 'subm_30',
    'submissions_count_365_days': 'subm_365',
    
    'submissions_count_contest'         : 'subm_c_all',
    'submissions_count_contest_1_days'  : 'subm_c_1',
    'submissions_count_contest_7_days'  : 'subm_c_7',
    'submissions_count_contest_30_days' : 'subm_c_30',
    'submissions_count_contest_365_days': 'subm_c_365',
    
    'submissions_count_master'         : 'subm_m_all',
    'submissions_count_master_1_days'  : 'subm_m_1',
    'submissions_count_master_7_days'  : 'subm_m_7',
    'submissions_count_master_30_days' : 'subm_m_30',
    'submissions_count_master_365_days': 'subm_m_365',
    
    'ipn_count'         : 'ipn_all',
    'ipn_count_1_days'  : 'ipn_1',
    'ipn_count_7_days'  : 'ipn_7',
    'ipn_count_30_days' : 'ipn_30',
    'ipn_count_365_days': 'ipn_365',
    
    'ipn_read'         : 'ipnr_all',
    'ipn_read_1_days'  : 'ipnr_1',
    'ipn_read_7_days'  : 'ipnr_7',
    'ipn_read_30_days' : 'ipnr_30',
    'ipn_read_365_days': 'ipnr_365',
    
    'forum_comments_count' : 'forum_reply',
    'forum_count'          : 'forum_cnt',
    'forum_expert_count'   : 'forum_exp',
    'forum_questions_count': 'forum_quest',
    'hacker_confirmation'  : 'confirmed',
    'hacker_timezone'      : 'timezone',
    'mail_category'        : 'mail_cat'
}

In [9]:
df_1.rename(columns=rename_dict, inplace=True)
df_2.rename(columns=rename_dict, inplace=True)

In [10]:
order = [
    'user_id', 'age_sent', 'last_seen', 'mail_id', 'mail_cat', 'timezone', 'confirmed',
    
    'forum_reply', 'forum_cnt', 'forum_exp', 'forum_quest',
    
    'clc_all', 'clc_1', 'clc_7', 'clc_30', 'clc_365',
    'cpc_all', 'cpc_1', 'cpc_7', 'cpc_30', 'cpc_365',
    'subm_all', 'subm_1', 'subm_7', 'subm_30', 'subm_365',
    'subm_c_all', 'subm_c_1', 'subm_c_7', 'subm_c_30', 'subm_c_365',
    'subm_m_all', 'subm_m_1', 'subm_m_7', 'subm_m_30', 'subm_m_365',
    'ipn_all', 'ipn_1', 'ipn_7', 'ipn_30', 'ipn_365',
    'ipnr_all', 'ipnr_1', 'ipnr_7', 'ipnr_30', 'ipnr_365',
]

In [11]:
df_1 = df_1[order + ['opened']]
df_2 = df_2[order]

----

Now almost everything is ready to investigate the dataset. Do not forget to check whether the user_id propagate hidden information. 

```
    user_1 = set(df_1['user_id'].unique())
    user_2 = set(df_2['user_id'].unique())
```

To recap, here is a list of categorical values with their cardinality:
 - user_id **30538**
 - mail_id **164**
 - mail_cat **18**
 - timezone **23**
 
I will ignore user_id for now, will truncate other categories to exclude everything that is less than 1%

In [12]:
def truncate_categorical(field, num):
    tmp = df_1[field].value_counts()
    vals = set(tmp[tmp < num].index.values)

    df_1[field][df_1[field].isin(vals)] = -1
    return vals

In [13]:
truncate_categorical('mail_id', 400);
truncate_categorical('timezone', 400);

Get rid of Null values.

In [14]:
df_1['mail_cat'].fillna(df_1['mail_cat'].value_counts().index.values[-1], inplace=True) # the least popular category
df_1['timezone'].fillna(-1, inplace=True) # group of least popular timezones
df_1['last_seen'].fillna(df_1['last_seen'].mean(), inplace=True) # mean value

In [15]:
df_1['mail_cat'] = df_1['mail_cat'].astype(int)
df_1['timezone'] = df_1['timezone'].astype(int)

In [16]:
df_y = df_1['opened']
df_1.drop(['opened'], axis=1, inplace=True)

------
## Trying to do ML

In [17]:
def analyse_many(estimators, X_train, y_train, X_test, y_test):
    all_values = []
    for estimator, name in estimators:
        startTime = datetime.now()
        score = f1(y_test, estimator.fit(X_train, y_train).predict(X_test))
        time_delta = datetime.now() - startTime
        all_values.append((score, name, time_delta.total_seconds()))
        
        print name, '\n  ', score, '\t', time_delta
    
    clear_output()
    all_values.sort(reverse=True)
    return pd.DataFrame(all_values, columns=['Score', 'Name', 'Time seconds']).set_index(['Score'])

In [18]:
estimators_new = []
for clf, name in [
    (linear_model.LogisticRegression,   'Logistic balanced'),
    (linear_model.RidgeClassifier,      'Ridge balanced'),
]:
    for w in xrange(710, 820, 5):
        estimators_new.append((clf(class_weight={1: w, 0: 1000 - w}), name + ' ' + str(w)))

In [19]:
estimators = [
    (linear_model.PassiveAggressiveClassifier(class_weight='balanced'),    'Passive aggressive'),
    (linear_model.SGDClassifier(class_weight='balanced'),                  'SGD'),
 
    (ensemble.RandomForestClassifier(class_weight='balanced'),             'Random forest'),
    (ensemble.AdaBoostClassifier(),                                        'Ada Boost'),
    (ensemble.GradientBoostingClassifier(),                                'Gradient Boosting'),
    (ensemble.BaggingClassifier(),                                         'Bagging'),
    (ensemble.ExtraTreesClassifier(class_weight='balanced'),               'Extra tree')
]

#### First attempt

In [20]:
def standard_flow(df):
    df_x = pd.get_dummies(df, columns=['mail_id', 'mail_cat', 'timezone'])

    X = df_x.values[:,1:]
    Y = df_y.values

    min_max_scaler = preprocessing.MinMaxScaler()
    X = min_max_scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, Y, test_size=0.40, random_state=0)
    
    return (
        analyse_many(estimators_new, X_train, y_train, X_test, y_test),
        analyse_many(estimators,     X_train, y_train, X_test, y_test),
    )

In [21]:
score_1, score_1_all = standard_flow(df_1)

SGD balanced had a reasonable results peaking at `0.507926591882 SGD balanced 807`.

Logistic and Ridge performed the best in the region from 700 to 800. Maximum is **0.519678**, achived with **Logistic balanced 744** (in a region from 735 to 745)

Tried the same without minMax scaler resulted in **6 times longer** processing and the maximum performance dropped very slightly.


#### Second attempt

Now I will try to cap some of the series:

In [22]:
df_1[df_1['age_sent'] >= df_1['age_sent'].quantile(0.9)] = df_1['age_sent'].quantile(0.98)
data_arr = [
    ('last_seen', 400, 500),
    ('forum_reply', 80, 100),
    ('forum_cnt', 40, 60),
    ('forum_exp', 5, 7),
    ('forum_quest', 5, 7),
    ('clc_all', 40, 60),
    ('clc_365', 40, 60),
    ('cpc_all', 100, 120),
    ('cpc_7', 20, 30),
    ('cpc_30', 50, 60),
    ('cpc_365', 100, 110),
    ('subm_1', 40, 60),
    ('subm_c_1', 40, 50),
    ('subm_m_1', 40, 50)
]
for name, max_val, cap_val in data_arr:
    df_1[df_1[name] >= max_val] = cap_val

def log_transform(x):
    return 0 if x == 0 else np.log2(x) + 1
    
for name in [
    'subm_all', 'subm_30', 'subm_365', 'subm_c_all', 'subm_c_7', 'subm_7', 'subm_c_365', 'subm_c_30',
    'subm_m_all', 'subm_m_7', 'subm_m_30', 'subm_m_365'
]:
    df_1[name] = df_1[name].apply(log_transform)

In [23]:
score_2, score_2_all = standard_flow(df_1)

The best achieved score is **0.516699** with `Logistic balanced 744`. It is slightly worse than the starting score.

-------
#### Third attempt

Now trying to get rid of `ipnr_*` `ipn_*`

In [24]:
df_1_copy = df_1.copy()

df_1_copy['ipn_all_percent'] = (df_1_copy['ipnr_all'] / df_1_copy['ipn_all']).fillna(0)
df_1_copy['ipn_1_percent']   = (df_1_copy['ipnr_1']   / df_1_copy['ipn_1']).fillna(0)
df_1_copy['ipn_7_percent']   = (df_1_copy['ipnr_7']   / df_1_copy['ipn_7']).fillna(0)
df_1_copy['ipn_30_percent']  = (df_1_copy['ipnr_30']  / df_1_copy['ipn_30']).fillna(0)
df_1_copy['ipn_365_percent'] = (df_1_copy['ipnr_365'] / df_1_copy['ipn_365']).fillna(0)

df_1_copy.drop(['ipnr_all', 'ipn_all', 'ipnr_1', 'ipn_1', 'ipnr_7', 'ipn_7', 'ipnr_30', 'ipn_30', 'ipnr_365', 'ipn_365'], axis=1, inplace=True)
    
score_3, score_3_all = standard_flow(df_1_copy)

#### Forth attempt

To use `fillna(0.5)`

In [25]:
df_1['ipn_all_percent'] = (df_1['ipnr_all'] / df_1['ipn_all']).fillna(0.5)
df_1['ipn_1_percent']   = (df_1['ipnr_1']   / df_1['ipn_1']).fillna(0.5)
df_1['ipn_7_percent']   = (df_1['ipnr_7']   / df_1['ipn_7']).fillna(0.5)
df_1['ipn_30_percent']  = (df_1['ipnr_30']  / df_1['ipn_30']).fillna(0.5)
df_1['ipn_365_percent'] = (df_1['ipnr_365'] / df_1['ipn_365']).fillna(0.5)

df_1.drop(['ipnr_all', 'ipn_all', 'ipnr_1', 'ipn_1', 'ipnr_7', 'ipn_7', 'ipnr_30', 'ipn_30', 'ipnr_365', 'ipn_365'], axis=1, inplace=True)

In [26]:
score_4, score_4_all = standard_flow(df_1)

#### Fifth attempt

remove a lot of columns

In [27]:
df_1.drop([
    'clc_all', 'clc_30', 'clc_365',
    'cpc_all', 'cpc_30', 'cpc_365',
    'subm_all', 'subm_30', 'subm_365',
    'subm_c_all', 'subm_c_30', 'subm_c_365',
    'subm_m_all', 'subm_m_30', 'subm_m_365'
], axis=1, inplace=True)
    
score_5, score_5_all = standard_flow(df_1)

For truncate_categorical (100, 300):
 - all values **0.519454**
 - capping series **0.518768**
 - got rid of `ipnr_*` `ipn_*`, filling with 0 **0.516237**
 - got rid of `ipnr_*` `ipn_*`, filling with 0.5 **0.518523**
 - remove many columns **0.516990**
 
For truncate categorical (400, 300). Number uniques (106, 22):
 - all values **0.519506**
 - capping series **0.518668**
 - got rid of `ipnr_*` `ipn_*`, filling with 0 **0.516095**
 - got rid of `ipnr_*` `ipn_*`, filling with 0.5 **0.518402**
 - remove many columns **0.516817**
 
For truncate categorical (600, 600). Number uniques (86, 21):
 - all values **0.519513**
 - capping series **0.518545**
 - got rid of `ipnr_*` `ipn_*`, filling with 0 **0.516087**
 - got rid of `ipnr_*` `ipn_*`, filling with 0.5 **0.518440**
 - remove many columns **0.516732**
 
Best PCA reduction technique achieved **0.513982**

In [28]:
pd.concat([
    score_1.head(1),
    score_2.head(1),
    score_3.head(1),
    score_4.head(1),
    score_5.head(1),
])

,Name,Time seconds
Score,,
0.520189,Logistic balanced 730,29.789122
0.519671,Logistic balanced 745,70.282320
0.517143,Logistic balanced 750,49.974155
0.519915,Logistic balanced 745,43.636953
0.518200,Logistic balanced 740,28.724288
